# Team Members

| **Name**                     | **ID**      | **Department**      |
|:----------------------------:|:-----------:|:-------------------:|
| Ahmed Ashraf                 | 2022446758  | Business            |
| Abdelrhman Mohamed Abdelhady | 2022513643  | Intelligent Systems |
| Antonuose Gerges Nageh       | 20221903971 | Intelligent Systems |

# Data Structures used

- Node `custom made structure`
- Tree `custom made structure`
- ArrayList 
- Queue
- Stack
- Priority Queue
- HashTable
- Enumeration
- ActionPath `custom made structure`

## Action Enumeration
It's used to represent that action taken by the parent node to reach the child node and it contains the following

- `Up`
- `Down`
- `Left`
- `Right`

## Creating the Node
The Node will contain the following data fields:

- `Parent:Node`
- `Children:ArrayList`
- `state:int[][]`
- `stringState:String` *which will be used to generate a hash code and compare it the goal state*
- `Direction:Action` *Action taken to reach this node*
- `depth:int`
- `missingTileRow:int`
- `missingTileCol:int`
- `cost:int`

It contains the following methods:

- `CreateStringBoard():String` which creates a string of the state of the node
- `addChild(Node child):void` which is a helper function when creating a child`
- `CreateChild(int a, int b):void` Creates a child where paramaters a,b represent the position of missing tile
- `getRowCol(int value):int[]` returns an array of size two, the method is used in calculating the herustic functions
- `equals(Object obj):boolean` `Override` It overrides the default equals method
- `isGoal():boolean` checks if the Node state is the goal state
- `hashCode():int` `Override` overrides the default hashCode method so that it returns the hash code of the string state
- `toString:String` `Override` Overrides the default toString method and makes use of the string builder object

## Creating the ActionPath class
The main goal of this class is to use backtracking the print the path from the goal node to the root node with the neccessary information.
The class uses the Stack data-structure *LIFO* where we are going to use the property that the last element that enters the stack is the first element that exits.

The methods the class contains are as follows:

- `getPath`:`Stack`
- `printPath`:`void`

`getpath` functions takes both the root node and goal node as inputs and pushes the goal node parents inside it till it reaches the root node.

`printPath` function returns nothing, it is only used for printing the path from the root to the goal.

## Creating the Tree

The tree class will contain the search functions as well as the expand function for the nodes.

Moreover, it contains the $f_1$ & $f_2$ Comparator objects that will be used in the priority queue.

It has only one data field : `root:Node`

The inner classes:

- `f_1` which implements Comparator used to determine whic node should enter the priorty queue first, It uses the manhattan distance herustic
- `f_2` has the same functionality as `f_1` but uses the Euclidean distance herustic

The methods:

- `expand(Node node):List<Node>` returns a list of the children of the given node i.e. it expands it
- `breadthFirstSearch():boolean`  returns true if the search is successful otherwise return false
- `depthFirstSearch():boolean` returns true if the search is successful otherwise return false
- `manhattanDistance(Node n):int` which calculates the manhattan distance
- `euclideanDistance(Node n):int` which calculates the euclidean distance
- `aStar(int i):boolean` which implements the A* star search, i paramter is used to determine which herustic to use


In [1]:
public enum Action {
    Up,Down,Left,Right
}


In [2]:
import java.util.ArrayList;
import java.util.List;

/*
 * Our node for the board needs
 * - a parent
 * - a state (a description of this state as a string for the hash table)
 * - children
 * - position of missing tile
 * - Action taken to reach this node
 * */
public class Node {
    // Data fields
    private Node parent;
    private List<Node> children;
    private int[][] state;
    private String stringState;
    private Action direction;
    private int depth = 0, missingTileRow, missingTileCol, cost;

    // Constructor
    public Node(int[][] state) {
        this.state = state;
        this.stringState = createStringBoard();
        this.parent = null;
        this.direction = null;
        this.children = new ArrayList<>();
        for (int i = 0; i < 3; i++) {
            for (int j = 0; j < 3; j++) {
                if (state[i][j] == 0) {
                    missingTileRow = i;
                    missingTileCol = j;
                    break;
                }
            }
        }
    }

    // Methods
    /*بتحول ال matrix ل string عشان المقرنة بعدين*/
    public String createStringBoard() {
        StringBuilder sb = new StringBuilder();
        for (int i = 0; i < 3; i++) {
            for (int j = 0; j < 3; j++)
                sb.append(this.state[i][j]);
        }
        return sb.toString();
    }

    // Add child function which will act as a helper function later
    /*داله مساعدة عشان لما نيجي تعمل ال child ل ال node*/
    public void addChild(Node child) {
        child.setParent(this);
        child.setDepth(this.depth + 1);
        child.setCost(this.cost + 1);
        this.children.add(child);
    }

    /*
     * Create a child which is the main way that we will use to actually add children with a state
     * a,b represent new position of missing tile
     * It returns a node which will be used later in expand fn
     * */
    /*
    * ال a و ال b مكان الصفرة الجديدة
    * بترجع node عشان نقدر نستعملها في ال expand */
    public Node createChild(int a, int b) {
        int[][] placeholder = new int[3][3];
        // copying the state matrix into the placeholder and switching the missing tile
        for (int i = 0; i < 3; i++)
            System.arraycopy(this.state[i], 0, placeholder[i], 0, 3);
        placeholder[missingTileRow][missingTileCol] = placeholder[a][b];
        placeholder[a][b] = 0;
        Node child = new Node(placeholder);
        child.setCost(1);
        this.addChild(child); // using the helper fn created earlier
        return child;
    }

    /*بترجع لنا مكان قيمة معينه في ال matrix
    * المكان بيرجع علي هيئة array
    * 0 -> س
    * 1 -> ص*/
    public int[] getRowCol(int value) {
        int[] container = new int[2];
        for (int i = 0; i < 3; i++) {
            for (int j = 0; j < 3; j++) {
                if (state[i][j] == value) {
                    container[0] = i;
                    container[1] = j;
                    break;
                }
            }
        }
        return container;
    }

    @Override
    public boolean equals(Object obj) {
        if (!(obj instanceof Node checker)) {
            return false;
        }
        return checker.getStringState().equals(this.getStringState());
    }

    public boolean isGoal() {
        int[][] goalState = {{0, 1, 2}, {3, 4, 5}, {6, 7, 8}};
        Node GoalState = new Node(goalState);
        return this.equals(GoalState);
    }

    /*بيحول ال String ل كود عشان لما تنجي نعمل جدول تحتفظة فيه بال node اللي شوفناها قبل كدة*/
    @Override
    public int hashCode() {
        return this.stringState.hashCode();
    }

    // getters
    public String getStringState() {
        return stringState;
    }

    public int getMissingTileRow() {
        return missingTileRow;
    }

    public int getMissingTileCol() {
        return missingTileCol;
    }

    public int getCost() {
        return cost;
    }

    public int getDepth() {
        return depth;
    }

    public Node getParent() {
        return parent;
    }

    public Action getDirection() {
        return direction;
    }

    // Setters
    public void setParent(Node parent) {
        this.parent = parent;
    }

    public void setDepth(int depth) {
        this.depth = depth;
    }

    public void setCost(int cost) {
        this.cost = cost;
    }

    public void setDirection(Action direction) {
        this.direction = direction;
    }


    @Override
    public String toString() {
        StringBuilder sb = new StringBuilder();
        for (int i = 0; i < 3; i++) {
            for (int j = 0; j < 3; j++) {
                sb.append(state[i][j]).append('\t');
            }
            sb.append("\n");
        }
        return sb.toString();
    }


}

In [3]:
import java.util.Stack;

public class ActionPath {
    // Data field
    Stack<Node> path = new Stack<>();

    // Constructor
    public ActionPath(Node initialNode, Node goalNode) {
        path = getPath(initialNode, goalNode);
    }

    // Methods
    public Stack<Node> getPath(Node initialNode, Node goalNode) {
        Node temp = goalNode;
        Stack<Node> list = new Stack<>();
        while (!temp.equals(initialNode)) {
            list.push(temp);
            temp = temp.getParent();
        }
        list.push(initialNode);
        return list;
    }

    public void printPath() {
        Node node = path.pop();
        System.out.println("The root node");
        System.out.print(node);
        while (path.size() > 0) {
            node = path.pop();
            System.out.println("-----------------------");
            System.out.println("Current Node: \n");
            System.out.println(node);
            System.out.println("Direction Moved: " + node.getDirection());
            System.out.println("Depth: " + node.getDepth());
            System.out.println("Cost: " + node.getCost());
        }
    }
}

In [4]:
import java.util.*;

public class Tree {
    Node root;

    public Tree(int[][] initialState) {
        root = new Node(initialState);
    }

    // Expand function
    public List<Node> expand(Node node) {
        int row = node.getMissingTileRow(), col = node.getMissingTileCol();
        List<Node> list = new ArrayList<>();
        // Left Action
        if (col != 0) {
            Node leftNode = node.createChild(row, col - 1);
            leftNode.setDirection(Action.Left);
            list.add(leftNode);
        }
        // Right Action
        if (col != 2) {
            Node rightNode = node.createChild(row, col + 1);
            rightNode.setDirection(Action.Right);
            list.add(rightNode);
        }
        // Up Action
        if (row != 0) {
            Node upNode = node.createChild(row - 1, col);
            upNode.setDirection(Action.Up);
            list.add(upNode);
        }
        // Down Action
        if (row != 2) {
            Node downNode = node.createChild(row + 1, col);
            downNode.setDirection(Action.Down);
            list.add(downNode);
        }
        return list;
    }


    public boolean depthFirstSearch() {
        double startTime = System.currentTimeMillis();
        int size = 0;
        Stack<Node> frontier = new Stack<>();
        HashMap<Integer, Node> reached = new HashMap<>();
        if (root.isGoal()) {
            double endTime = System.currentTimeMillis();
            ActionPath path = new ActionPath(root, root);
            path.printPath();
            System.out.println("-----------------------");
            System.out.println("Time: " + (endTime - startTime) + " millie seconds");
            System.out.println("Space: " + size);
            return true;
        }
        frontier.add(root);
        size++;
        reached.put(root.hashCode(), root);
        while (!(frontier.isEmpty())) {
            Node node = frontier.pop();
            for (Node child : expand(node)) {
                if (child.isGoal()) {
                    double endTime = System.currentTimeMillis();
                    size += 1;
                    ActionPath path = new ActionPath(root, child);
                    path.printPath();
                    System.out.println("-----------------------");
                    System.out.println("Time: " + (endTime - startTime) + " millie seconds");
                    System.out.println("Space: " + size);
                    return true;
                }
                if (!(reached.containsKey(child.hashCode())) && !(frontier.contains(child))) {
                    frontier.push(child);
                    reached.put(child.hashCode(), child);
                    size += 1;
                }
            }
        }

        System.out.println("Time: " + (System.currentTimeMillis() - startTime) + " millie seconds");
        System.out.println("Space: " + size);
        return false;
    }

    public boolean breadthFirstSearch() {
        double startTime = System.currentTimeMillis();
        int size = 0;
        Queue<Node> frontier = new LinkedList<>();
        HashMap<Integer, Node> reached = new HashMap<>();
        if (root.isGoal()) {
            size++;
            double endTime = System.currentTimeMillis();
            ActionPath path = new ActionPath(root, root);
            path.printPath();
            System.out.println("-----------------------");
            System.out.println("Time: " + (endTime - startTime) + " millie seconds");
            System.out.println("Space: " + size);
            return true;
        }
        frontier.add(root);
        size++;
        reached.put(root.hashCode(), root);
        while (!(frontier.isEmpty())) {
            Node node = frontier.poll();
            for (Node child : expand(node)) {
                if (child.isGoal()) {
                    double endTime = System.currentTimeMillis();
                    size += 1;
                    ActionPath path = new ActionPath(root, child);
                    path.printPath();
                    System.out.println("-----------------------");
                    System.out.println("Time: " + (endTime - startTime) + " millie seconds");
                    System.out.println("Space: " + size);
                    return true;
                }
                if (!(reached.containsKey(child.hashCode())) && !(frontier.contains(child))) {
                    frontier.add(child);
                    reached.put(child.hashCode(), child);
                    size += 1;
                }
            }
        }
        System.out.println("Time: " + (System.currentTimeMillis() - startTime) + " millie seconds");
        System.out.println("Space: " + size);
        return false;
    }

    private int manhattanDistance(Node n) {
        int[][] goalState = {{0, 1, 2}, {3, 4, 5}, {6, 7, 8}};
        int h = 0;
        for (int i = 0; i < 3; i++) {
            for (int j = 0; j < 3; j++) {
                int[] container = n.getRowCol(goalState[i][j]);
                h += Math.abs(i - container[0]) + Math.abs(j - container[1]);
            }
        }
        return h;
    }

    private int euclideanDistance(Node n) {
        int[][] goalState = {{0, 1, 2}, {3, 4, 5}, {6, 7, 8}};
        int h = 0;
        for (int i = 0; i < 3; i++) {
            for (int j = 0; j < 3; j++) {
                int[] container = n.getRowCol(goalState[i][j]);
                h += Math.sqrt((i - container[0]) * (i - container[0]) + Math.abs(j - container[1]) * Math.abs(j - container[1]));
            }
        }
        return h;
    }

    private class f_1 implements Comparator<Node> {

        @Override
        public int compare(Node o1, Node o2) {
            return (manhattanDistance(o1) + o1.getCost()) - (manhattanDistance(o2) + o2.getCost());
        }
    }

    private class f_2 implements Comparator<Node> {

        @Override
        public int compare(Node o1, Node o2) {
            return (euclideanDistance(o1) + o1.getCost()) - (euclideanDistance(o2) + o2.getCost());
        }
    }

    public boolean aStar(int i) {
        double startTime = System.currentTimeMillis();
        PriorityQueue<Node> frontier;
        if (i == 1) {
            frontier = new PriorityQueue<>(new f_1());
        } else {
            frontier = new PriorityQueue<>(new f_2());
        }
        int size = 0;
        HashMap<Integer, Node> reached = new HashMap<>();
        if (root.isGoal()) {
            size++;
            double endTime = System.currentTimeMillis();
            ActionPath path = new ActionPath(root, root);
            path.printPath();
            System.out.println("-----------------------");
            System.out.println("Time: " + (endTime - startTime) + " millie seconds");
            System.out.println("Space: " + size);
            return true;
        }
        frontier.add(root);
        size++;
        reached.put(root.hashCode(), root);
        while (!(frontier.isEmpty())) {
            Node node = frontier.poll();
            for (Node child : expand(node)) {
                if (child.isGoal()) {
                    double endTime = System.currentTimeMillis();
                    size += 1;
                    ActionPath path = new ActionPath(root, child);
                    path.printPath();
                    System.out.println("-----------------------");
                    System.out.println("Time: " + (endTime - startTime) + " millie seconds");
                    System.out.println("Space: " + size);
                    return true;
                }
                if (!(reached.containsKey(child.hashCode())) && !(frontier.contains(child))) {
                    frontier.add(child);
                    reached.put(child.hashCode(), child);
                    size += 1;
                }
            }
        }
        System.out.println("Time: " + (System.currentTimeMillis() - startTime) + " millie seconds");
        System.out.println("Space: " + size);
        return false;
    }
}

# Breadth First Search : 1 2 3 4 0 5 6 7 8
Testing the Breadth first search.

Breadth frist search was implemented with an early goal test.

In [5]:
import java.util.Scanner;
public static void failure() {
    System.out.println("The puzzle you have entered is unsolvable and resulted in failure");
}

Scanner sc = new Scanner(System.in);
        int[][] initialState = new int[3][3];
        int input1, input2;
        System.out.println("Welcome to 8 puzzle Solver");
        System.out.print("Enter the puzzle : ");
        boolean inputCorrect = false;
        do {
            int[] freq = new int[9];
            for (int i = 0; i < 3; i++) {
                for (int j = 0; j < 3; j++) {
                    initialState[i][j] = sc.nextInt();
                    if (initialState[i][j] > 8)
                        break;
                    freq[initialState[i][j]]++;
                }
            }
            sc.nextLine();
            for (int i = 0; i < 9; i++) {
                if (freq[i] != 1) {
                    inputCorrect = false;
                    System.out.println("The input you have entered is incorrect. Please try again!");
                    System.out.print("Enter the puzzle : ");
                    break;
                }
                inputCorrect = true;
            }
        } while (!inputCorrect);
        Tree Board = new Tree(initialState);
        System.out.println("Choose the Algorithm");
        System.out.println();
        System.out.println("1. BFS");
        System.out.println("2. DFS");
        System.out.println("3. A*");
        System.out.println();
        System.out.print("Enter your choice: ");
        input1 = sc.nextInt();

        switch (input1) {
            case 1 -> {
                if (!(Board.breadthFirstSearch())) failure();
            }
            case 2 -> {
                if (!Board.depthFirstSearch()) failure();
            }
            default -> {
                System.out.println("Choose the Heuristic function");
                System.out.println();
                System.out.println("1. Manhattan Distance");
                System.out.println("2. Euclidean Distance");
                System.out.println();
                System.out.print("Enter your choice: ");
                input2 = sc.nextInt();
                if (input2 == 1) {
                    if (!Board.aStar(1)) failure();
                } else {
                    if (!(Board.aStar(2))) failure();
                }
            }
        }

Welcome to 8 puzzle Solver
Enter the puzzle : 1 2 3 4 0 5 6 7 8
Choose the Algorithm

1. BFS
2. DFS
3. A*

Enter your choice: 1
The root node
1	2	3	
4	0	5	
6	7	8	
-----------------------
Current Node: 

1	2	3	
0	4	5	
6	7	8	

Direction Moved: Left
Depth: 1
Cost: 1
-----------------------
Current Node: 

0	2	3	
1	4	5	
6	7	8	

Direction Moved: Up
Depth: 2
Cost: 2
-----------------------
Current Node: 

2	0	3	
1	4	5	
6	7	8	

Direction Moved: Right
Depth: 3
Cost: 3
-----------------------
Current Node: 

2	3	0	
1	4	5	
6	7	8	

Direction Moved: Right
Depth: 4
Cost: 4
-----------------------
Current Node: 

2	3	5	
1	4	0	
6	7	8	

Direction Moved: Down
Depth: 5
Cost: 5
-----------------------
Current Node: 

2	3	5	
1	0	4	
6	7	8	

Direction Moved: Left
Depth: 6
Cost: 6
-----------------------
Current Node: 

2	0	5	
1	3	4	
6	7	8	

Direction Moved: Up
Depth: 7
Cost: 7
-----------------------
Current Node: 

0	2	5	
1	3	4	
6	7	8	

Direction Moved: Left
Depth: 8
Cost: 8
-----------------------
Current

# Depth First Search  : 1 0 2 3 4 5 6 7 8
The implementation contains an early goal test even though that might not result in the optimal solution but it results in better search time.

In [6]:
import java.util.Scanner;
public static void failure() {
    System.out.println("The puzzle you have entered is unsolvable and resulted in failure");
}

Scanner sc = new Scanner(System.in);
        int[][] initialState = new int[3][3];
        int input1, input2;
        System.out.println("Welcome to 8 puzzle Solver");
        System.out.print("Enter the puzzle : ");
        boolean inputCorrect = false;
        do {
            int[] freq = new int[9];
            for (int i = 0; i < 3; i++) {
                for (int j = 0; j < 3; j++) {
                    initialState[i][j] = sc.nextInt();
                    if (initialState[i][j] > 8)
                        break;
                    freq[initialState[i][j]]++;
                }
            }
            sc.nextLine();
            for (int i = 0; i < 9; i++) {
                if (freq[i] != 1) {
                    inputCorrect = false;
                    System.out.println("The input you have entered is incorrect. Please try again!");
                    System.out.print("Enter the puzzle : ");
                    break;
                }
                inputCorrect = true;
            }
        } while (!inputCorrect);
        Tree Board = new Tree(initialState);
        System.out.println("Choose the Algorithm");
        System.out.println();
        System.out.println("1. BFS");
        System.out.println("2. DFS");
        System.out.println("3. A*");
        System.out.println();
        System.out.print("Enter your choice: ");
        input1 = sc.nextInt();

        switch (input1) {
            case 1 -> {
                if (!(Board.breadthFirstSearch())) failure();
            }
            case 2 -> {
                if (!Board.depthFirstSearch()) failure();
            }
            default -> {
                System.out.println("Choose the Heuristic function");
                System.out.println();
                System.out.println("1. Manhattan Distance");
                System.out.println("2. Euclidean Distance");
                System.out.println();
                System.out.print("Enter your choice: ");
                input2 = sc.nextInt();
                if (input2 == 1) {
                    if (!Board.aStar(1)) failure();
                } else {
                    if (!(Board.aStar(2))) failure();
                }
            }
        }

Welcome to 8 puzzle Solver
Enter the puzzle : 1 0 2 3 4 5 6 7 8
Choose the Algorithm

1. BFS
2. DFS
3. A*

Enter your choice: 2
The root node
1	0	2	
3	4	5	
6	7	8	
-----------------------
Current Node: 

0	1	2	
3	4	5	
6	7	8	

Direction Moved: Left
Depth: 1
Cost: 1
-----------------------
Time: 0.0 millie seconds
Space: 2


# A* search
## Manhattan distance - puzzle : 1 2 3 4 5 6 7 8 0
The first Search -Manhatten distance- using the following puzzle : 1 2 3 4 5 6 7 8 0


In [7]:
import java.util.Scanner;
public static void failure() {
    System.out.println("The puzzle you have entered is unsolvable and resulted in failure");
}

Scanner sc = new Scanner(System.in);
        int[][] initialState = new int[3][3];
        int input1, input2;
        System.out.println("Welcome to 8 puzzle Solver");
        System.out.print("Enter the puzzle : ");
        boolean inputCorrect = false;
        do {
            int[] freq = new int[9];
            for (int i = 0; i < 3; i++) {
                for (int j = 0; j < 3; j++) {
                    initialState[i][j] = sc.nextInt();
                    if (initialState[i][j] > 8)
                        break;
                    freq[initialState[i][j]]++;
                }
            }
            sc.nextLine();
            for (int i = 0; i < 9; i++) {
                if (freq[i] != 1) {
                    inputCorrect = false;
                    System.out.println("The input you have entered is incorrect. Please try again!");
                    System.out.print("Enter the puzzle : ");
                    break;
                }
                inputCorrect = true;
            }
        } while (!inputCorrect);
        Tree Board = new Tree(initialState);
        System.out.println("Choose the Algorithm");
        System.out.println();
        System.out.println("1. BFS");
        System.out.println("2. DFS");
        System.out.println("3. A*");
        System.out.println();
        System.out.print("Enter your choice: ");
        input1 = sc.nextInt();

        switch (input1) {
            case 1 -> {
                if (!(Board.breadthFirstSearch())) failure();
            }
            case 2 -> {
                if (!Board.depthFirstSearch()) failure();
            }
            default -> {
                System.out.println("Choose the Heuristic function");
                System.out.println();
                System.out.println("1. Manhattan Distance");
                System.out.println("2. Euclidean Distance");
                System.out.println();
                System.out.print("Enter your choice: ");
                input2 = sc.nextInt();
                if (input2 == 1) {
                    if (!Board.aStar(1)) failure();
                } else {
                    if (!(Board.aStar(2))) failure();
                }
            }
        }

Welcome to 8 puzzle Solver
Enter the puzzle : 1 2 3 4 5 6 7 8 0
Choose the Algorithm

1. BFS
2. DFS
3. A*

Enter your choice: 3
Choose the Heuristic function

1. Manhattan Distance
2. Euclidean Distance

Enter your choice: 1
The root node
1	2	3	
4	5	6	
7	8	0	
-----------------------
Current Node: 

1	2	3	
4	5	6	
7	0	8	

Direction Moved: Left
Depth: 1
Cost: 1
-----------------------
Current Node: 

1	2	3	
4	0	6	
7	5	8	

Direction Moved: Up
Depth: 2
Cost: 2
-----------------------
Current Node: 

1	2	3	
4	6	0	
7	5	8	

Direction Moved: Right
Depth: 3
Cost: 3
-----------------------
Current Node: 

1	2	0	
4	6	3	
7	5	8	

Direction Moved: Up
Depth: 4
Cost: 4
-----------------------
Current Node: 

1	0	2	
4	6	3	
7	5	8	

Direction Moved: Left
Depth: 5
Cost: 5
-----------------------
Current Node: 

0	1	2	
4	6	3	
7	5	8	

Direction Moved: Left
Depth: 6
Cost: 6
-----------------------
Current Node: 

4	1	2	
0	6	3	
7	5	8	

Direction Moved: Down
Depth: 7
Cost: 7
-----------------------
Current Node

## Euclidean distance - puzzle : 1 2 3 4 5 6 7 8 0
The second search uses the ecludiean distance to find goal node. It uses the same puzzle as the manhattan herustic

In [8]:
import java.util.Scanner;
public static void failure() {
    System.out.println("The puzzle you have entered is unsolvable and resulted in failure");
}
Scanner sc = new Scanner(System.in);
        int[][] initialState = new int[3][3];
        int input1, input2;
        System.out.println("Welcome to 8 puzzle Solver");
        System.out.print("Enter the puzzle : ");
        boolean inputCorrect = false;
        do {
            int[] freq = new int[9];
            for (int i = 0; i < 3; i++) {
                for (int j = 0; j < 3; j++) {
                    initialState[i][j] = sc.nextInt();
                    if (initialState[i][j] > 8)
                        break;
                    freq[initialState[i][j]]++;
                }
            }
            sc.nextLine();
            for (int i = 0; i < 9; i++) {
                if (freq[i] != 1) {
                    inputCorrect = false;
                    System.out.println("The input you have entered is incorrect. Please try again!");
                    System.out.print("Enter the puzzle : ");
                    break;
                }
                inputCorrect = true;
            }
        } while (!inputCorrect);
        Tree Board = new Tree(initialState);
        System.out.println("Choose the Algorithm");
        System.out.println();
        System.out.println("1. BFS");
        System.out.println("2. DFS");
        System.out.println("3. A*");
        System.out.println();
        System.out.print("Enter your choice: ");
        input1 = sc.nextInt();

        switch (input1) {
            case 1 -> {
                if (!(Board.breadthFirstSearch())) failure();
            }
            case 2 -> {
                if (!Board.depthFirstSearch()) failure();
            }
            default -> {
                System.out.println("Choose the Heuristic function");
                System.out.println();
                System.out.println("1. Manhattan Distance");
                System.out.println("2. Euclidean Distance");
                System.out.println();
                System.out.print("Enter your choice: ");
                input2 = sc.nextInt();
                if (input2 == 1) {
                    if (!Board.aStar(1)) failure();
                } else {
                    if (!(Board.aStar(2))) failure();
                }
            }
        }

Welcome to 8 puzzle Solver
Enter the puzzle : 1 2 3 4 5 6 7 8 0
Choose the Algorithm

1. BFS
2. DFS
3. A*

Enter your choice: 3
Choose the Heuristic function

1. Manhattan Distance
2. Euclidean Distance

Enter your choice: 2
The root node
1	2	3	
4	5	6	
7	8	0	
-----------------------
Current Node: 

1	2	3	
4	5	6	
7	0	8	

Direction Moved: Left
Depth: 1
Cost: 1
-----------------------
Current Node: 

1	2	3	
4	0	6	
7	5	8	

Direction Moved: Up
Depth: 2
Cost: 2
-----------------------
Current Node: 

1	2	3	
4	6	0	
7	5	8	

Direction Moved: Right
Depth: 3
Cost: 3
-----------------------
Current Node: 

1	2	0	
4	6	3	
7	5	8	

Direction Moved: Up
Depth: 4
Cost: 4
-----------------------
Current Node: 

1	0	2	
4	6	3	
7	5	8	

Direction Moved: Left
Depth: 5
Cost: 5
-----------------------
Current Node: 

0	1	2	
4	6	3	
7	5	8	

Direction Moved: Left
Depth: 6
Cost: 6
-----------------------
Current Node: 

4	1	2	
0	6	3	
7	5	8	

Direction Moved: Down
Depth: 7
Cost: 7
-----------------------
Current Node